# Convert .laz to .las

In [1]:
import numpy as np

# Read the PCD file into a NumPy array
data = np.fromfile("/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/060_2-295_2_slam.pcd", dtype=np.float32)

# Print the number of points in the PCD file
print(data.shape[0])

# Print the coordinates of the first point in the PCD file
print(data[0])
print(data.shape)

57160247
11685367000.0
(57160247,)


In [2]:

import os, sys
sys.path.append("../../")
from src.forest3D.pcd import read_pcd


path = "/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/060_2-295_2_slam.pcd"
# Import the Pandas dataframe.
pcd_file = read_pcd(path)
xyz_data_pcd = pcd_file['points'][['x','y','z']].values


In [ ]:
import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,ground_removal,treeDetector,inventory_tools,utilities

import os
import json
import numpy as np

output_dir = '../../data/'

# print("reading the data into python...")
# #path = '/home/lloyd/Documents/datasets/lidar/forestry_usyd/extracts/plots_for_training/V1_Scanner1_161011_220153_crop026.asc'
# #xyz_data = lidar_IO.XYZreadFromCSV(path,delimiter=' ',x=0,y=1,z=2)
# #path = '/home/lloyd/Documents/datasets/lidar/forestry_usyd/FCNSW_FertTrialAOI_VUX1_conv.las'
# path = '../../data/Forest.las'
# xyz_data = lidar_IO.readFromLas(path, fields = ['x','y','z'])

path = "/forest_3d_app/3d_forest/forest_3d_app/data/pcd_files/060_2-295_2_slam.pcd"
# Import the Pandas dataframe.
pcd_file = read_pcd(path)
xyz_data = pcd_file['points'][['x','y','z']].values


print("removing the ground...")
offset = [0,0,0]       # optional: use to make calculations stable on large values
xyz_data_gr = ground_removal.removeGround(xyz_data,offset,thresh=2.0,proc_path=output_dir)
ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir,'_ground_surface.ply'))


detector_addr = '../../models/models/detection/tumut1/'
with open(os.path.join(detector_addr, 'raster_config.json')) as json_file:
    config_dict = json.load(json_file)
rasterTreeDetector = treeDetector.RasterDetector(**config_dict )
labels = rasterTreeDetector.sliding_window(detector_addr,xyz_data_gr,ground_pts=ground_pts,windowSize = [600,600],stepSize = 80)



print("outputing detection results...")
lidar_IO.writePly_labelled(os.path.join(output_dir,'detection.ply'),xyz_data_gr,labels,offset)


print("outputting inventory...")
tree_tops = inventory_tools.get_tree_tops(xyz_data_gr,labels)
heights = inventory_tools.get_tree_heights(tree_tops[:,:3],ground_pts)
inventory = np.hstack((tree_tops,heights[:,np.newaxis]))
utilities.write_csv(os.path.join(output_dir,'inventory.csv'),inventory,header='x,y,z,id,height')

removing the ground...
28 out of 40 tiles

In [8]:
labels

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
tree_tops

array([[ 1.09716049e+02,  7.13374405e+01,  1.87170506e+01,
         1.00000000e+00],
       [ 8.32406693e+01,  6.32035685e+00,  1.43260593e+01,
         2.00000000e+00],
       [ 6.61739960e+01, -7.35914040e+00,  1.32201529e+01,
         3.00000000e+00],
       [ 4.28680916e+01,  1.97724289e+02,  1.78835049e+01,
         4.00000000e+00],
       [ 9.69813538e+01,  5.09190865e+01,  2.82280426e+01,
         5.00000000e+00],
       [ 1.10105850e+02,  3.59764938e+01,  2.97579632e+01,
         6.00000000e+00],
       [ 5.79293137e+01,  2.07114975e+02,  2.16382179e+01,
         7.00000000e+00],
       [ 2.97466965e+01,  1.13540359e+02,  2.00105705e+01,
         8.00000000e+00],
       [-2.12286377e+00,  1.97613541e+02,  2.43312664e+01,
         9.00000000e+00],
       [ 2.43083153e+01,  2.21410889e+02,  2.00958385e+01,
         1.00000000e+01],
       [ 9.37241440e+01,  6.65171890e+01,  1.83791294e+01,
         1.10000000e+01],
       [ 3.05220203e+01, -2.00284538e+01,  1.17006817e+01,
      

In [11]:
xyz_data_pcd.values

array([[  2.874758  ,  33.740757  ,  -0.9702503 ],
       [  0.22088976,   8.482922  ,  -2.8069754 ],
       [  1.3441689 ,  12.568088  ,  -1.5623947 ],
       ...,
       [ 38.83735   , 187.37492   ,   7.377514  ],
       [ 51.076355  , 195.37656   ,  13.019906  ],
       [ 54.94717   , 194.85497   ,   9.85992   ]], dtype=float32)

In [10]:
xyz_data_las

array([[ 34.36,  14.9 ,   2.19],
       [ 17.68, -18.62,   6.11],
       [-13.72,   1.52,   4.96],
       ...,
       [ 18.66, -18.56,   6.42],
       [ 20.22, -19.33,   7.04],
       [ 20.27, -19.48,   7.07]])